In [18]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable Weights & Biases logging

import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
df = pd.read_csv("D:\Projects\BERT\merged_output.csv")


<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Aranya Jana\AppData\Local\Temp\ipykernel_16632\3407470904.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_csv("D:\Projects\BERT\merged_output.csv")


In [20]:
print("columns:")
print(df.columns.tolist())

columns:
['text', 'hatespeech', 'offensive', 'race', 'religion', 'gender', 'sexual', 'misc', 'age', 'not_cyberbullying']


In [22]:

# Fill missing values and define cyberbullying label
label_cols = ['hatespeech', 'offensive', 'race', 'religion', 'gender', 'sexual', 'misc', 'age', 'not_cyberbullying']
df[label_cols] = df[label_cols].fillna(0)
df['cyberbullying'] = df[label_cols].drop(columns=['not_cyberbullying']).max(axis=1)

# Prepare texts and labels
texts = df['text'].astype(str).tolist()
labels = df['cyberbullying'].astype(int).tolist()

In [23]:

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

In [24]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [26]:
# Dataset class
class CyberbullyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = CyberbullyDataset(train_encodings, train_labels)
test_dataset = CyberbullyDataset(test_encodings, test_labels)


In [28]:
# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

NameError: name 'init_empty_weights' is not defined

In [16]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_dir='./logs',
    logging_steps=10
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'